In [28]:
import nnsight
from nnsight import LanguageModel
from nnsight.intervention import InterventionProxy
from typing import List, Optional, Tuple, Union
from rich import print as rprint
from rich.table import Table

In [19]:
model = LanguageModel('gpt2')
tokenizer = model.tokenizer

In [21]:
print(f'Layers: {model.config.n_layer}, Resid stream dim: {model.config.n_embd}')

Layers: 12, Resid stream dim: 768


In [22]:
tokens = model.generate("At the funeral she said solemnly:", max_new_tokens=20)
tokens

In [33]:
REMOTE = False

In [41]:
sampling_kwargs = {
    "do_sample": True,
    "top_p": 0.3,
    "repetition_penalty": 1.1,
}

def calculate_and_apply_steering_vector(
    model: LanguageModel,
    prompt: str,
    activation_additions: List[Tuple[int, float, str]],
    n_tokens: int,
    n_comparisons: int = 1,
    use_bos: bool = True,
) -> Tuple[List[str], List[str]]:
    '''
    Performs the steering vector experiments described in the LessWrong post.

    Args:
        prompt: str
            The original prompt, which we'll be doing activation steering on.

        activation_additions: List[Tuple[int, float, str]], each tuple contains:
            layer - the layer we're applying these steering vectors to
            coefficient - the value we're multiplying it by
            prompt - the prompt we're inputting
            e.g. activation_additions[0] = [6, 5.0, " Love"] means we add the " Love" vector at layer 6, scaled by 5x

        n_tokens: int
            Number of tokens which will be generated for each completion

        n_comparisons: int
            Number of sequences generated in this function (i.e. we generate `n_comparisons` which are unsteered, and
            the same number which are steered).

    Returns:
        unsteered_completions: List[str]
            List of length `n_comparisons`, containing all the unsteered completions.

        steered_completions: List[str]
            List of length `n_comparisons`, containing all the steered completions.
    '''
    # Add the BOS token manually, if we're including it
    if use_bos:
        bos = model.tokenizer.bos_token
        prompt = bos + prompt
        activation_additions = [[layer, coeff, bos + p] for layer, coeff, p in activation_additions]

    # Get the (layers, coeffs, prompts) in an easier form to use, also calculate the prompt lengths & check they're all the same
    act_add_layers, act_add_coeffs, act_add_prompts = zip(*activation_additions)
    act_add_seq_lens = [len(tokenizer.tokenize(p)) for p in act_add_prompts]
    assert len(set(act_add_seq_lens)) == 1, "All activation addition prompts must be the same length."
    assert act_add_seq_lens[0] <= len(tokenizer.tokenize(prompt)), "All act_add prompts should be shorter than original prompt."

    # Get the prompts we'll intervene on (unsteered and steered)
    steered_prompts = [prompt for _ in range(n_comparisons)]
    unsteered_prompts = [prompt for _ in range(n_comparisons)]

    with model.generate(max_new_tokens=n_tokens, remote=False, remote_include_output=True, **sampling_kwargs) as generator:

        # Run the act_add prompts (i.e. the contrast pairs), and extract their activations
        with generator.invoke(act_add_prompts) as invoker:
            # Get all the prompts from the activation additions, and put them in a list
            # (note, we slice from the end of the sequence because of left-padding)
            act_add_vectors = [
                model.transformer.h[layer].output[0][i, -seq_len:]
                for i, (layer, seq_len) in enumerate(zip(act_add_layers, act_add_seq_lens))
            ]

        # Forward pass on unsteered prompts (no intervention, no activations saved - we only need the completions)
        with generator.invoke(unsteered_prompts) as invoker:
            pass

        # Forward pass on steered prompts (we add in the results from the act_add prompts)
        with generator.invoke(steered_prompts) as invoker:
            # For each act_add prompt, add the vector to residual stream, at the start of the sequence
            for i, (layer, coeff, seq_len) in enumerate(zip(act_add_layers, act_add_coeffs, act_add_seq_lens)):
                model.transformer.h[layer].output[0][:, :seq_len] += act_add_vectors[i] * coeff

    # Decode steered & unsteered completions (discarding the sequences we only used for extracting activations) & return results
    unsteered_completions = tokenizer.batch_decode(generator.output[-2*n_comparisons: -n_comparisons])
    steered_completions = tokenizer.batch_decode(generator.output[-n_comparisons:])
    return unsteered_completions, steered_completions

In [44]:
unsteered_completions, steered_completions = calculate_and_apply_steering_vector(
    model,
    prompt = "I went up to my friend and said",
    activation_additions = [
        (10, +6.0, "I talk about cookies constantly  "),
        (10, -6.0, "I do not talk about cookies constantly"),
    ],
    n_tokens = 50,
    n_comparisons = 3,
    use_bos = False,
)

table = Table("Unsteered", "Steered", title="Completions", show_lines=True)
for usc, sc in zip(unsteered_completions, steered_completions):
    table.add_row(usc, sc)
rprint(table)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


                                                    Completions                                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Unsteered                                              ┃ Steered                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ I went up to my friend and said, 'Hey I've got a new   │ I went up to my friend and said, 'You know what? I'm   │
│ book coming out. What do you think?' He was like,      │ going back in time.' And he was like, 'Yeah. That's    │
│ 'Well it's really good.'                               │ how it goes.'"                                         │
│ "He told me he'd read the first two books in his life  │  (The cookie-cutter cookies were the Cookie of The     │
│ so that would be an interesting thing                  │ Year.) "He told me that his first thought              │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ I went up to my friend and said, 'What's wrong with    │ I went up to my friend and said, 'What are you talking │
│ you?' And he was like, 'Oh no. I'm not going back.' "  │ about?' He was like, 'You know what cookie I'm talkin' │
│  (Photo: AP) Story Highlights A former student at the  │ on.' And he's got a real good understanding of         │
│ University of Wisconsin-Madison says she is now a      │ cookies. So we were just kinda sitting there thinking  │
│ victim in her                                          │ that maybe it would be better                          │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ I went up to my friend and said, 'Hey buddy. I'm going │ I went up to my friend and said, 'You know what? I'm   │
│ over there with you.' And he was like, 'Yeah? What's   │ going to go out with you.' And he was like, 'No.       │
│ that?' He had a lot of fun talking about it."          │ You're not.'"                                          │
│  (Photo: Courtesy) Story Highlights The first time the │ - The Cookie Monster's cookie is a candy that can be   │
│ couple met in                                          │ used as an ice cream or for the cookies of             │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

In [47]:
unsteered_completions, steered_completions = calculate_and_apply_steering_vector(
    model,
    prompt = "I hate you because",
    activation_additions = [
        (11, +5.0, "Love "),
        (11, -5.0, "Hate"),
    ],
    n_tokens = 50,
    n_comparisons = 3,
    use_bos = True,
)

table = Table("Unsteered", "Steered", title="Completions", show_lines=True)
for usc, sc in zip(unsteered_completions, steered_completions):
    table.add_row(usc, sc)
rprint(table)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


                                                    Completions                                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Unsteered                                              ┃ Steered                                                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ <|endoftext|>I hate you because I'm a racist.          │ <|endoftext|>I hate you because I love my life.        │
│ It's not that they're bad people, it's just how we are │                                                        │
│ and what our culture is about them being so good at    │ It's been a long time since we've had the chance to    │
│ hating us as well. We have to stop doing this shit for │ talk about our lives and what it means to be human,    │
│ ourselves or the world around                          │ but now that she has spoken out against her husband    │
│                                                        │ for not being able take care of his                    │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ <|endoftext|>I hate you because I love your guts.      │ <|endoftext|>I hate you because I'm a man.             │
│                                                        │                                                        │
│ And that's why, in the end of my life and for many     │ But that's not what this is about. This is an argument │
│ years after it happened, we were all so happy to have  │ for the right to be free from oppression and           │
│ him back on our side as a friend who was just about    │ exploitation, which means we have freedom of speech on │
│ ready enough to get out                                │ all sides. We can say whatever we want in our          │
├────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────┤
│ <|endoftext|>I hate you because I don't like your      │ <|endoftext|>I hate you because I love your work.      │
│ politics.                                              │                                                        │
│                                                        │ But when it comes to the art of making money, this is  │
│ But that's not the point of this article. It is a      │ what makes me happy: my passion for music and its      │
│ simple question: Why do we need to have more people in │ effects on people's lives has been a source that keeps │
│ our country? The answer lies somewhere between         │ coming back into focus as time goes by                 │
│ political correctness and "political Islamophobia."    │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

In [51]:
unsteered_completions, steered_completions = calculate_and_apply_steering_vector(
    model,
    prompt = "To see the eiffel tower, people flock to",
    activation_additions = [
        (9, +10.0, "The Eiffel Tower is in Rome"),
        (9, -10.0, "The Eiffel Tower is in France"),
    ],
    n_tokens = 50,
    n_comparisons = 3,
    use_bos = False,
)

table = Table("Unsteered", "Steered", title="Completions", show_lines=True)
for usc, sc in zip(unsteered_completions, steered_completions):
    table.add_row(usc, sc)
rprint(table)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
